### Forecast Suitable Zipcodes for Starbucks locations :

In [1]:
#%matplotlib notebook
import zipfile
import gmaps
import gmaps.datasets
import os
import pandas as pd
import kaggle
import requests
import json
from census import Census
import matplotlib.pyplot as plt
import requests
import json
from pygeocoder import Geocoder, GeocoderError
from pyzipcode import ZipCodeDatabase
import scipy.stats as stats

# Census API Key
from config import census_key, gkey
c = Census(census_key, year=2013)
gmaps.configure(api_key=gkey)

#url for google geocode api
basegeo_url = "https://maps.googleapis.com/maps/api/geocode/json"

### Download Starbucks dataset from kaggle & uncompress it

In [2]:
#Retrive the dataset from kaggle and move to Resources folder
os.system('kaggle datasets download -d starbucks/store-locations' )
os.system('mv -f ./store-locations.zip ./Resources/')

0

In [3]:
#Unzip the dataset downloaded from kaggle
zip_file = os.path.join('Resources', 'store-locations.zip' )
zip_ref = zipfile.ZipFile(zip_file, 'r')
zip_ref.extractall('Resources')
zip_ref.close()

In [4]:
#Read starbucks file into a dataframe
str_file = os.path.join('Resources','directory.csv')
str_df = pd.read_csv(str_file)
str_df.head(5)

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [5]:
#Create a dataframe for starbucks in USA 
str_df_us = str_df.loc[str_df['Country'] == 'US']
str_df_us.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
11964,Starbucks,3513-125945,Safeway-Anchorage #1809,Licensed,5600 Debarr Rd Ste 9,Anchorage,AK,US,995042300,907-339-0900,GMT-09:00 America/Anchorage,-149.78,61.21
11965,Starbucks,74352-84449,Safeway-Anchorage #2628,Licensed,1725 Abbott Rd,Anchorage,AK,US,995073444,907-339-2800,GMT-09:00 America/Anchorage,-149.84,61.14
11966,Starbucks,12449-152385,Safeway - Anchorage #1813,Licensed,1501 Huffman Rd,Anchorage,AK,US,995153596,907-339-1300,GMT-09:00 America/Anchorage,-149.85,61.11
11967,Starbucks,24936-233524,100th & C St - Anchorage,Company Owned,"320 W. 100th Ave, 100, Southgate Shopping Ctr ...",Anchorage,AK,US,99515,(907) 227-9631,GMT-09:00 America/Anchorage,-149.89,61.13
11968,Starbucks,8973-85630,Old Seward & Diamond,Company Owned,1005 E Dimond Blvd,Anchorage,AK,US,995152050,907-344-4160,GMT-09:00 America/Anchorage,-149.86,61.14


### Create new column - Zipcode from existing Postalcode

In [6]:
#Add a new column - Zipcode & populate it from Postcode
str_df_us['Zipcode'] = '00000'
str_df_us.Zipcode = str_df_us.Postcode.astype(str).str[0:5]
str_df_us.count()

C:\Users\rz0001\AppData\Local\Continuum\anaconda3\envs\Pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rz0001\AppData\Local\Continuum\anaconda3\envs\Pythondata\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Brand             13608
Store Number      13608
Store Name        13608
Ownership Type    13608
Street Address    13608
City              13608
State/Province    13608
Country           13608
Postcode          13607
Phone Number      13122
Timezone          13608
Longitude         13608
Latitude          13608
Zipcode           13608
dtype: int64

### Data Cleansing - populate missing postalcode using address from Google Geocode

In [7]:
#Get Zipcodes from Google Geocode API for records without a Poscode.
for index, row in str_df_us.iterrows():
    if (  str(   (row)['Postcode']   )  == 'nan'  ):
        #print(index, row)
        address = row['Street Address'] + ' ' + row['State/Province'] + ' ' + row['Country']
        params = { 'address' : address  , 'key' : gkey }
        #print(address)
        response = requests.get(basegeo_url, params=params).json()
        pc = response['results'][0]['address_components'][7]['short_name']
        str_df_us.loc[str_df_us['Store Number'] == row['Store Number'], 'Zipcode'] = pc

C:\Users\rz0001\AppData\Local\Continuum\anaconda3\envs\Pythondata\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Download census data

In [8]:
#Get census data
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_002E": "Employment Labor Force",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Employment Labor Force", "Poverty Rate"]]

census_pd.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Employment Labor Force,Poverty Rate
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3220.0,3.258578
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,16187.0,6.676860
2,08525,4782.0,47.1,118656.0,59848.0,43.0,3004.0,0.899205
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,29204.0,3.993293
4,08528,245.0,48.5,58676.0,49117.0,0.0,130.0,0.000000


In [9]:
#Merge Starbucks and Census dataframes on Zipcode
data_df = census_pd.merge(str_df_us, how = 'outer', on="Zipcode")
data_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Employment Labor Force,Poverty Rate,Brand,Store Number,...,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3220.0,3.258578,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,16187.0,6.676860,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,08525,4782.0,47.1,118656.0,59848.0,43.0,3004.0,0.899205,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,29204.0,3.993293,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,08528,245.0,48.5,58676.0,49117.0,0.0,130.0,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleansing - Standardize Brandname

In [10]:
# Data Analysis - standardizing names for Starbucks

data_df['Brand'] = data_df['Brand'].str.strip().replace("Teavana","Starbucks")
data_df['Brand'] = data_df['Brand'].str.strip().replace("Evolution Fresh","Starbucks")
data_df['Brand'] = data_df['Brand'].str.strip().replace("Coffee House Holdings","Starbucks")

### Write File to Resources folder for Analysis

In [14]:
write_file = os.path.join('Resources' , 'SB_Census_Data.csv')
data_df.to_csv(write_file)